### Libraries:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import pad_sequences
import tensorflow as tf
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import numpy as np
from sklearn.dummy import DummyRegressor
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense

### Warnings:

In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The objective has been evaluated at this point before.")

In [3]:
from google.colab import files
uploaded = files.upload()

Saving data1_filled.csv to data1_filled.csv


In [5]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['data1_filled.csv']))
df

,Study_Folder,Subject_Folder,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,0,235.0,195.0,204.0,204.0,269.0,377.0,269.0,296.0,0.469347,0.124451,0.166405
1,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,1,235.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.469335,0.124451,0.166405
2,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,2,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.465500,0.124451,0.166405
3,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,3,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.477129,0.124451,0.166405
4,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,4,235.0,195.0,203.0,203.0,269.0,377.0,268.0,296.0,0.469347,0.124451,0.166405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107845,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,200,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527,0.124735,0.721234
107846,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,201,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992,0.223992,0.539739
107847,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,202,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993,0.124735,0.733096
107848,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,203,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297,0.124204,0.717675


### Read dataset:

In [6]:
# df = pd.read_csv('data1_filled.csv')
# df

### Group by Study_Folder:

In [7]:
ispitanci_X = []
ispitanici_y = []
for s, df_s in df.groupby(['Study_Folder']):
    train_mask = df_s['Stimuli'] < 10
    valid_mask = (df_s['Stimuli'] >= 10) & (df_s['Stimuli'] <= 11)
    test_mask = df_s['Stimuli'] > 11
    features = df_s.columns[4:13]
    ys = df_s.columns[13:15]
    X_tr, X_v, X_te = df_s[train_mask][features], df_s[valid_mask][features], df_s[test_mask][features]
    y_tr, y_v, y_te = df_s[train_mask][ys], df_s[valid_mask][ys], df_s[test_mask][ys]

    ispitanci_X.append([X_tr, X_v, X_te])
    ispitanici_y.append([y_tr, y_v, y_te])

<ipython-input-7-c61f4bbed505>:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for s, df_s in df.groupby(['Study_Folder']):


### Metrics:

In [8]:
metric_dict = {"MAE":mean_absolute_error, "MSE":mean_squared_error, "R2": r2_score}

### Log table:

In [9]:
log_table_total =[]

### Dummy Regressor:

In [10]:
dummy_regr = DummyRegressor(strategy="mean")
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go modelot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    dummy_regr.fit(X_tr, y_tr)
    y_pred_dr=dummy_regr.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['DummyRegressor', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_dr)])

100%|██████████| 38/38 [00:00<00:00, 253.13it/s]


In [11]:
n_steps = 3
input_features = 9
n_units = 50

### Function for creating sequences:

In [12]:
def create_sequences(data_X, data_y, n_steps_in=3): # many to many:
    m,n = [],[]
    for i in range(data_X.shape[0]-n_steps_in):
        m.append(data_X.values[i:i+n_steps]); n.append(data_y.values[i+n_steps])
    return np.array(m), np.array(n)

### SimpleRNN + LSTM + GRU:

In [ ]:
model_2 = Sequential()
model_2.add(SimpleRNN(200, activation = 'relu', return_sequences = True, input_shape = (n_steps, input_features)))
model_2.add(LSTM(n_units, activation = 'relu', return_sequences = True))
model_2.add(GRU(n_units, activation = 'tanh'))
model_2.add(Dense(2, activation = 'linear'))
model_2.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_2.summary()

In [14]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go model_2ot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences(X_v, y_v, n_steps)
    X_te, y_te = create_sequences(X_te, y_te, n_steps)

    model_2.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=10, batch_size = 32, verbose = 0)
    y_pred_te = model_2.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['SimpleRNN+LSTM+GRU', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/38 [00:00<?, ?it/s]

29/29 [==============================] - 0s 2ms/step


  3%|▎         | 1/38 [00:06<04:01,  6.54s/it]

48/48 [==============================] - 0s 2ms/step


  5%|▌         | 2/38 [00:09<02:34,  4.29s/it]

...

 95%|█████████▍| 36/38 [01:52<00:04,  2.43s/it]

51/51 [==============================] - 0s 2ms/step


 97%|█████████▋| 37/38 [01:54<00:02,  2.54s/it]

52/52 [==============================] - 0s 2ms/step


100%|██████████| 38/38 [02:00<00:00,  3.16s/it]


### Many to many (3 rows for train, predict on 2):

In [15]:
n_steps = 3
input_features = 9
n_steps_out=2
n_units = 50

In [16]:
def create_sequences_1(data_X, data_y, n_steps_in=3,n_steps_out=2): # many to many:
    m,n = [],[]
    for i in range(data_X.shape[0]-n_steps_in-n_steps_out+1):
        m.append(data_X.values[i:i+n_steps])
        new_y=data_y.values[i+n_steps:i+n_steps_in+n_steps_out].flatten()
        n.append(new_y)
    return np.array(m), np.array(n)

In [ ]:
model_3 = Sequential()
model_3.add(SimpleRNN(n_units, activation = 'relu', return_sequences = True, input_shape = (n_steps, input_features)))
model_3.add(LSTM(100, activation = 'relu', return_sequences = True))
model_3.add(GRU(n_units, activation = 'relu'))
model_3.add(Dense(n_steps_out*2, activation = 'linear'))
model_3.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_3.summary()

In [18]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go modelot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences_1(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences_1(X_v, y_v, n_steps)
    X_te, y_te = create_sequences_1(X_te, y_te, n_steps)


    model_3.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=10, batch_size = 32, verbose = 0)
    y_pred_te = model_3.predict(X_te)

    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['SimpleRNN+LSTM+GRU_many_to_many', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/38 [00:00<?, ?it/s]

29/29 [==============================] - 0s 2ms/step


  3%|▎         | 1/38 [00:05<03:31,  5.71s/it]

48/48 [==============================] - 0s 2ms/step


  5%|▌         | 2/38 [00:08<02:23,  4.00s/it]

...

 95%|█████████▍| 36/38 [01:59<00:05,  2.54s/it]

51/51 [==============================] - 0s 2ms/step


 97%|█████████▋| 37/38 [02:01<00:02,  2.62s/it]

52/52 [==============================] - 0s 2ms/step


100%|██████████| 38/38 [02:07<00:00,  3.35s/it]


### Conv1D + SimpleRNN + LSTM + GRU:

In [ ]:
model_4 = Sequential()
model_4.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(3, 9)))
model_4.add(SimpleRNN(n_units, activation = 'relu', return_sequences = True, input_shape = (n_steps, input_features)))
model_4.add(LSTM(100, activation = 'relu', return_sequences = True))
model_4.add(GRU(n_units, activation = 'sigmoid'))
model_4.add(Dense(2, activation = 'linear'))
model_4.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_4.summary()

In [20]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go model_2ot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences(X_v, y_v, n_steps)
    X_te, y_te = create_sequences(X_te, y_te, n_steps)

    model_4.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=10, batch_size = 32, verbose = 0)
    y_pred_te = model_4.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['Conv1D&SimpleRNN&LSTM&GRU', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/38 [00:00<?, ?it/s]

29/29 [==============================] - 0s 1ms/step


  3%|▎         | 1/38 [00:05<03:32,  5.75s/it]

48/48 [==============================] - 0s 1ms/step


  5%|▌         | 2/38 [00:08<02:24,  4.02s/it]

...

 95%|█████████▍| 36/38 [01:16<00:03,  1.75s/it]

51/51 [==============================] - 0s 1ms/step


 97%|█████████▋| 37/38 [01:18<00:02,  2.05s/it]

52/52 [==============================] - 0s 1ms/step


100%|██████████| 38/38 [01:21<00:00,  2.15s/it]


### Conv1D + LSTM + GRU

In [ ]:
model_6 = Sequential()
model_6.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(3, 9)))
model_6.add(LSTM(100, activation = 'relu',input_shape = (n_steps, input_features), return_sequences = True))
model_6.add(GRU(n_units, activation = 'relu'))
model_6.add(Dense(2, activation = 'linear'))
model_6.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_6.summary()

In [22]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go model_2ot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences(X_v, y_v, n_steps)
    X_te, y_te = create_sequences(X_te, y_te, n_steps)

    model_6.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=10, batch_size = 32, verbose = 0)
    y_pred_te = model_6.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['Conv1d+LSTM+GRU', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/38 [00:00<?, ?it/s]

29/29 [==============================] - 0s 1ms/step


  3%|▎         | 1/38 [00:03<02:24,  3.90s/it]

48/48 [==============================] - 0s 1ms/step


  5%|▌         | 2/38 [00:05<01:37,  2.71s/it]

...

 95%|█████████▍| 36/38 [01:17<00:04,  2.33s/it]

51/51 [==============================] - 0s 1ms/step


 97%|█████████▋| 37/38 [01:20<00:02,  2.48s/it]

52/52 [==============================] - 0s 1ms/step


100%|██████████| 38/38 [01:22<00:00,  2.16s/it]


### Conv1D+RNN+LSTM+GRU with tanh

In [23]:
model_8 = Sequential()
model_8.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(3, 9)))
model_8.add(SimpleRNN(n_units, activation = 'relu', return_sequences = True, input_shape = (n_steps, input_features)))
model_8.add(LSTM(200, activation = 'tanh', return_sequences = True))
model_8.add(GRU(100, activation = 'relu'))
model_8.add(Dense(2, activation = 'tanh'))
model_8.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_8.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 1, 16)             448       
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 1, 50)             3350      
                                                                 
 lstm_4 (LSTM)               (None, 1, 200)            200800    
                                                                 
 gru_4 (GRU)                 (None, 100)               90600     
                                                                 
 dense_4 (Dense)             (None, 2)                 202       
                                                                 
Total params: 295,400
Trainable params: 295,400
Non-trainable params: 0
_________________________________________________________________


In [24]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go model_2ot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences(X_v, y_v, n_steps)
    X_te, y_te = create_sequences(X_te, y_te, n_steps)

    model_8.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=20, batch_size = 32, verbose = 0)
    y_pred_te = model_8.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['tanh_Conv1D+RNN+LSTM+GRU', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/38 [00:00<?, ?it/s]

29/29 [==============================] - 1s 2ms/step


  3%|▎         | 1/38 [00:09<05:39,  9.17s/it]

48/48 [==============================] - 0s 2ms/step


  5%|▌         | 2/38 [00:19<05:57,  9.94s/it]

...

 95%|█████████▍| 36/38 [04:39<00:15,  7.73s/it]

51/51 [==============================] - 0s 2ms/step


 97%|█████████▋| 37/38 [04:47<00:07,  7.88s/it]

52/52 [==============================] - 0s 2ms/step


100%|██████████| 38/38 [04:58<00:00,  7.84s/it]


### Log table total:

In [25]:
log_table_total = pd.DataFrame(log_table_total, columns = ['Model','Ispitanik','Metric','Length of Test','Metric score'])

In [26]:
log_table_total

,Model,Ispitanik,Metric,Length of Test,Metric score
0,DummyRegressor,1,MAE,910,0.285679
1,DummyRegressor,1,MSE,910,0.082680
2,DummyRegressor,1,R2,910,-332.501899
3,DummyRegressor,2,MAE,1513,0.314344
4,DummyRegressor,2,MSE,1513,0.104856
...,...,...,...,...,...
679,tanh_Conv1D+RNN+LSTM+GRU,37,MSE,1632,0.105075
680,tanh_Conv1D+RNN+LSTM+GRU,37,R2,1632,-183786.132166
681,tanh_Conv1D+RNN+LSTM+GRU,38,MAE,1661,0.298142
682,tanh_Conv1D+RNN+LSTM+GRU,38,MSE,1661,0.100858


### Function for weighted - Mean

In [27]:
def calculate_weighted(m, log_table_total):
    num_frames = log_table_total[log_table_total['Metric'] == 'MSE']['Length of Test'].sum()
    results = []
    for d, gf in log_table_total.groupby('Metric'):
        adjusted_score = gf['Length of Test'] / num_frames * gf['Metric score']
        adjusted_score = adjusted_score.mean()
        results.append((m, d, adjusted_score))
    return results

In [28]:
final_table = []
for m, m_df in log_table_total.groupby('Model'):
    results = calculate_weighted(m, m_df)
    final_table.extend(results)

final_table_columns = ['Model', 'Metric', 'Adjusted Score']
final_table = pd.DataFrame(final_table, columns=final_table_columns)

### Final log table - Mean:

In [29]:
final_table

,Model,Metric,Adjusted Score
0,Conv1D&SimpleRNN&LSTM&GRU,MAE,0.006674
1,Conv1D&SimpleRNN&LSTM&GRU,MSE,0.002254
2,Conv1D&SimpleRNN&LSTM&GRU,R2,-602.034672
3,Conv1d+LSTM+GRU,MAE,0.006860
4,Conv1d+LSTM+GRU,MSE,0.002490
5,Conv1d+LSTM+GRU,R2,-620.835790
6,DummyRegressor,MAE,0.006715
7,DummyRegressor,MSE,0.002179
8,DummyRegressor,R2,-918.003925
9,SimpleRNN+LSTM+GRU,MAE,0.006749


### Save csv file results:

In [30]:
log_table_total.to_csv('log_table_total1.csv', index=False)

In [31]:
from google.colab import files

files.download('log_table_total1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
final_table.to_csv('final_table1.csv', index=False)

In [33]:
from google.colab import files

files.download('final_table1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>